<a href="https://colab.research.google.com/github/itssofiarce/sagrada/blob/main/codigo/video_transcript.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalaciones necesarias
!apt-get update
!pip install selenium webdriver-manager flask youtube-transcript-api
!pip install langdetect

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Rutas
archivo_salida_txt = "/content/drive/My Drive/NLP_Repo_Sagrada/datos/informacion/videos_transcriptos.txt"
url_videos = "https://boardgamegeek.com/boardgame/199561/sagrada/videos/all?pageid=1&sort=hot"

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import csv
import os

# Iniciar Selenium
def start_driver(url, delay=5):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    service = Service('/usr/bin/chromedriver')
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)
    print(f"Esperando {delay} segundos para que cargue la página...")
    time.sleep(delay)
    return driver

# Extraer vídeos
def get_youtube_video_ids_from_bgg(driver, url_base):
    driver.get(url_base)
    time.sleep(5)
    print("Cargando enlaces internos de videos...")

    # Buscar todos los enlaces a páginas de videos
    links = driver.find_elements(By.CSS_SELECTOR, 'a[href^="/video/"]')
    video_page_urls = list(set([link.get_attribute("href") for link in links]))

    print(f"Se encontraron {len(video_page_urls)} páginas de videos. Extrayendo enlaces de YouTube...")

    youtube_ids = []
    for video_url in video_page_urls:
        try:
            driver.get(video_url)
            time.sleep(3)
            iframe = driver.find_element(By.CSS_SELECTOR, 'iframe[src*="youtube.com/embed"]')
            youtube_src = iframe.get_attribute('src')
            video_id = youtube_src.split("/embed/")[-1].split("?")[0]
            titulo = driver.title
            youtube_ids.append((titulo, video_id, video_url))
            print(f"✔ {titulo} - ID: {video_id}")
        except Exception as e:
            print(f"✘ No se encontró video de YouTube en {video_url}: {e}")
    return youtube_ids


from youtube_transcript_api import YouTubeTranscriptApi
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound
from langdetect import detect

def get_transcripts(video_list, output_path):
    with open(output_path, "w", encoding="utf-8") as f:
        for titulo, video_id, url in video_list:
            try:
                transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['es', 'en'])
                text = "\n".join([seg["text"] for seg in transcript])
                detected_lang = detect(text)

                f.write(
                    f"TÍTULO: {titulo}\n"
                    f"ID: {video_id}\n"
                    f"URL: {url}\n"
                    f"IDIOMA DETECTADO: {detected_lang}\n"
                    f"TRANSCRIPCIÓN:\n{text}\n\n"
                    f"{'-'*60}\n\n"
                )

                print(f"✔ Transcripción guardada para video: {titulo} (Idioma: {detected_lang})")

            except (TranscriptsDisabled, NoTranscriptFound):
                print(f"✘ No se pudo obtener transcripción para video: {titulo}")
            except Exception as e:
                print(f"⚠ Error inesperado con el video {titulo}: {str(e)}")



In [7]:
# Ejecutar todo
driver = start_driver(url_videos)
# Extraer todos los video_ids desde páginas internas
video_info_list = get_youtube_video_ids_from_bgg(driver, url_videos)
# Cerrar Selenium
driver.quit()
# Transcribir vídeos
get_transcripts(video_info_list, archivo_salida_txt)

Esperando 5 segundos para que cargue la página...
Cargando enlaces internos de videos...
Se encontraron 37 páginas de videos. Extrayendo enlaces de YouTube...
✔ Sagrada - Unboxing | Video | BoardGameGeek - ID: eQOGVm1AvK0
✔ Gen Con Bonanza 2016: Sagrada Interview | Video | BoardGameGeek - ID: FRQEEpFyRoM
✔ [NTFG] Sagrada - Stained Glass | Video | BoardGameGeek - ID: FD_Eo22tplA
✔ Sagrada — Gen Con 2016 | Video | BoardGameGeek - ID: GMeu2GbirqI
✘ No se encontró video de YouTube en https://boardgamegeek.com/video/new/thing/199561: Message: no such element: Unable to locate element: {"method":"css selector","selector":"iframe[src*="youtube.com/embed"]"}
  (Session info: chrome=136.0.7103.49); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5ba4740077fa <unknown>
#1 0x5ba473aabe90 <unknown>
#2 0x5ba473afd870 <unknown>
#3 0x5ba473afda61 <unknown>
#4 0x5ba473b4bcb4 <unknown>
#5 0